In [1]:
from PIL import Image
#import tensorflow.keras.preprocessing.image as kpi
from random import randint
import numpy as np
import cv2
import matplotlib.pyplot as plt

Utilisation de la librairie PIL pour la création des images synthétiques par collages, les masques sont des images PNG de chardons.

###  Exemple avec une image

In [6]:
mask_path='mask/'
img_path='images_background/'
masking_insapath= 'Masking_insa/output/mask/'
masking= Image.open(masking_insapath+'background3.jpg')
colors = masking.getpixel((4055,3039))
print(colors)
background = Image.open(img_path+'background3.jpg' )
bg_w, bg_h = background.size
print(background.size)

255
(4056, 3040)


In [7]:
# Open overlay image
img = Image.open(mask_path+'mask1.png')

img_w, img_h = img.size
background = Image.open(img_path+'background1.jpg')
bg_w, bg_h = background.size
#offset = ((bg_w - img_w) // 2, (bg_h - img_h) // 2)
background.paste(img,masking)
background.save('result.png')
background.show()


ValueError: images do not match

### Fonction de génération

Les masques, les backgrounds ainsi que les positions des chardons sont sélectionnés aléatoirement.

In [8]:
def createxy(x,y):
    x_normal=4060
    y_normal=3050
    while (x_normal > 4055 or y_normal > 3039) :
        x_normal=np.random.normal(x,120)
        y_normal=np.random.normal(y,120)
    return x_normal,y_normal

In [9]:
def img_generation(mask_path,img_path,N):
    #mask_path='mask/'
    #img_path='images_background/'
    
    for i in range(0,N):     
        b_rand=randint(1,4)#randomiser l'image de fond
        
        nb_chardon=randint(8,10)#nombre de chardons
        nb_gangs=randint(1,4)
        #print(b_rand)
        background = Image.open(img_path+'background%d.jpg' %(b_rand))
        bg_w, bg_h = background.size
        
        file = open("generated_images/coordinates%d.txt" %(i),"a")
        file.truncate(0)
        for j in range(1):
            #background = Image.open(img_path+'background%d.jpg' %(b_rand))
            #bg_w, bg_h = background.size
            
            x_rand=randint(0,bg_w)#randomiser la position
            y_rand=randint(0,bg_h)#randomiser la position
            for k in range(1,nb_chardon):
                m_rand=randint(1,30)#randomiser le mask
                angle_rand=randint(0,360)#randomiser l'angle de rotation
                foreground = Image.open(mask_path+'mask%d.png' %(m_rand))
                foreground.rotate(angle_rand)
                masking_insa=Image.open(masking_insapath+'background%d.jpg' %(b_rand))
                x_normal=createxy(x_rand,y_rand)[0]
                y_normal=createxy(y_rand,y_rand)[1]
                while masking_insa.getpixel((int(x_normal),int(y_normal)))!=0 :
                    x_normal=createxy(x_rand,y_rand)[0]
                    y_normal=createxy(y_rand,y_rand)[1]
                m_w,m_h=foreground.size
                file.write(str(int(x_normal)) +" " +str(int(y_normal))+ " "+str(int(x_normal)+m_w) +" "
                +str(int(y_normal)+m_h)+"\n")

                background.paste(foreground, (int(x_normal), int(y_normal)),foreground)#retirer le "foreground" pour avoir le fond noir
        file.close()
        background.save('generated_images/generated%d.png' %(i))

In [5]:
img_generation(mask_path,img_path,10)

NameError: name 'mask_path' is not defined

## Poisson editing

les deux focntions suivantes servent à la création des masks et sources pour l'algorithme de poisson editing

In [10]:
def img_generation2(mask_path,img_path,N):
    #mask_path='mask/'
    #img_path='images_background/'
    
    for i in range(0,N):     
        b_rand=randint(1,4)#randomiser l'image de fond
        
        #nb_chardon=randint(8,10)#nombre de chardons
        nb_chardon=1
        nb_gangs=randint(2,3)
        #print(b_rand)
        background = Image.open(img_path+'background%d.jpg' %(b_rand))
        bg_w, bg_h = background.size
        bg = Image.new('RGB', (4056, 3040))
        file = open("generated_paste/coordinates%d.txt" %(i),"a")
        file.truncate(0)
        #for j in range(1,nb_gangs+1):
        for j in range(1):
            #background = Image.open(img_path+'background%d.jpg' %(b_rand))
            #bg_w, bg_h = background.size
            
            x_rand=randint(0,bg_w)#randomiser la position
            y_rand=randint(0,bg_h)#randomiser la position
            for k in range(nb_chardon):
                m_rand=randint(1,30)#randomiser le mask
                angle_rand=randint(0,360)#randomiser l'angle de rotation
                foreground = Image.open(mask_path+'mask%d.png' %(m_rand))
                foreground.rotate(angle_rand)
                masking_insa=Image.open(masking_insapath+'background%d.jpg' %(b_rand))
                x_normal=createxy(x_rand,y_rand)[0]
                y_normal=createxy(y_rand,y_rand)[1]
                while masking_insa.getpixel((int(x_normal),int(y_normal)))!=0 :
                    x_normal=createxy(x_rand,y_rand)[0]
                    y_normal=createxy(y_rand,y_rand)[1]
                m_w,m_h=foreground.size
                file.write(str(int(x_normal)) +" " +str(int(y_normal))+ " "+str(int(x_normal)+m_w) +" "
                +str(int(y_normal)+m_h)+"\n")
                
                
                bg.paste(foreground, (int(x_normal), int(y_normal)),foreground)#retirer le "foreground" pour avoir le fond noir
        file.close()
        bg.save('generated_paste/paste%d.png' %(i))

In [7]:
img_generation2(mask_path,img_path,10)

In [8]:
def mask_creation(N):
    for i in range(N):
        im = Image.open('generated_paste/paste%d.png' %(i))

        # Make Numpy array
        ni = np.array(im)

        
        reds=(ni[:,:,0]!=0)
        greens=(ni[:,:,1]!=0)
        blues = (ni[:,:,2]!=0)
        mask = np.logical_or(reds,greens,blues)
        # Save logical mask as PNG
        #Image.fromarray((blues*255).astype(np.uint8)).save('result.png')
        Image.fromarray((mask*255).astype(np.uint8)).save('generated_masks/mask%d.png' %(i))

In [12]:
        im = Image.open('paste_v3.png')

        # Make Numpy array
        ni = np.array(im)

        
        reds=(ni[:,:,0]!=0)
        greens=(ni[:,:,1]!=0)
        blues = (ni[:,:,2]!=0)
        mask = np.logical_or(reds,greens,blues)
        # Save logical mask as PNG
        #Image.fromarray((blues*255).astype(np.uint8)).save('result.png')
        Image.fromarray((mask*255).astype(np.uint8)).save('mask_v3.png')

In [9]:
mask_creation(10)

## Xml creation

In [13]:
import xml.etree.ElementTree as ET
import cv2
import numpy as np
import matplotlib.pyplot as plt

def get_annots_from_xml(path):
    """
    Récupère les annotations contenues dans un fichier .xml au format Pascal VOC.
    Retourne un array utilisable en entrée des modèles Tensorflow et des fonctions d'affichage.
    """
    tree = ET.parse(path)
    root = tree.getroot()
    
    size_x = int(root.findall('size')[0][0].text)
    size_y = int(root.findall('size')[0][1].text)
    objects_annot = root.findall('object')
    arr_box_annot = np.zeros((len(objects_annot), 4), dtype=np.float32)

    for i, box_annot in enumerate(root.findall('object')):
        arr_box_annot[i, :] = [int(box_annot[4][1].text) / size_y,
                               int(box_annot[4][0].text) / size_x,
                               int(box_annot[4][3].text) / size_y,
                               int(box_annot[4][2].text) / size_x]
    return arr_box_annot

def create_xml_from_detection(path_xml, detection_boxes, detection_scores,
                              seuil_detection, name_image, path_image, width, height, depth):
    """
    Génère un XML au format Pascal VOC contenant les annotations détectées avec une confiance supérieure au seuil.
    """
    # Création de la racine du document et des infos générales
    root = ET.Element("annotation")
    ET.SubElement(root, "folder").text = "patchs"
    ET.SubElement(root, "filename").text = str(name_image)
    ET.SubElement(root, "path").text = str(path_image)
    source = ET.SubElement(root, "source")
    ET.SubElement(source, "database").text = "Detection"
    size = ET.SubElement(root, "size")
    ET.SubElement(size, "width").text = str(width)
    ET.SubElement(size, "height").text = str(height)
    ET.SubElement(size, "depth").text = str(depth)
    ET.SubElement(root, "segmented").text = "0"
    
    # Nombre de bounding boxes détectées avec assez de confiance
    nb_boxes = np.sum(detection_scores >= seuil_detection)
    
    # Création de chaque bounding box
    # Le modèle renvoie les bounding boxes par ordre décroissant de confiance
    for i in range(nb_boxes):
        arr_boxes = np.array(detection_boxes)[i, :]
        obj = ET.SubElement(root, "object")
        ET.SubElement(obj, "name").text = "chardon"
        ET.SubElement(obj, "pose").text = "Unspecified"
        ET.SubElement(obj, "truncated").text = "0"
        ET.SubElement(obj, "difficult").text = "0"
        bndbox = ET.SubElement(obj, "bndbox")
        ET.SubElement(bndbox, "xmin").text = str(int(np.round(arr_boxes[1] * width)))
        ET.SubElement(bndbox, "ymin").text = str(int(np.round(arr_boxes[0] * height)))
        ET.SubElement(bndbox, "xmax").text = str(int(np.round(arr_boxes[3] * width)))
        ET.SubElement(bndbox, "ymax").text = str(int(np.round(arr_boxes[2] * height)))
                                                    
    # Assemblage de l'arbre
    tree = ET.ElementTree(root)
    
    # Ecriture du fichier
    tree.write(path_xml)

In [13]:
nb_images=10
for i in range(nb_images):
    annots=np.loadtxt("generated_images/coordinates%d.txt"%(i),"int")
    create_xml_from_detection("generated_images/generated%d.xml" %(i), annots, np.ones(annots.shape[0]),
                              0, "generated%d.jpg" %(i), "generated_images/generated%d.jpg" %(i), 1, 1, 3)

NameError: name 'create_xml_from_detection' is not defined

## create patches

In [17]:
HEIGHT_PATCH, WIDTH_PATCH = 400, 400


def load_image_into_numpy_array(path):
    img = cv2.cvtColor(cv2.imread(path, cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)
    return img